In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [2]:
%pwd

'/home/rui/Desktop/New Folder/text-summarization/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/rui/Desktop/New Folder/text-summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path : Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay : int
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [6]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config=read_yml(config_filepath)
        self.params=read_yml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params= self.params.TrainingArguments

        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" # if it crashes
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


/home/rui/miniconda3/envs/textS/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer:

    def __init__(self,config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # device = "cuda" if torch.cuda.is_available() else "cpu"
        device="cpu"
        tokenizer= AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator= DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)


        # loading data from this datapath
        dataset_samsum_pt= load_from_disk(self.config.data_path)

        # setting up training argument (reading from yaml file)
        # trainer_args= TrainingArguments(
        #     output_dir= self.config.root_dir,
        #     num_train_epochs=self.config.num_train_epochs,
        #     warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size,
        #     per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay,
        #     logging_steps=self.config.logging_steps,
        #     eval_strategy=self.config.evaluation_strategy,
        #     eval_steps=self.config.eval_steps,
        #     save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )


        # or read like hardcode way
        trainer_args=TrainingArguments(
            output_dir=self.config.root_dir,num_train_epochs=1,warmup_steps=500,
            per_device_train_batch_size=1,per_device_eval_batch_size=1,
            weight_decay=0.01,logging_steps=10,
            eval_strategy='steps',eval_steps=500,save_steps=1e6,
            gradient_accumulation_steps=16
        )

        trainer=Trainer(
            model=model_pegasus,args=trainer_args,
            tokenizer=tokenizer,data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"]
        )

        trainer.train()

        ## save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus_samsum_model"))

        ## save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config= config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-09-02 10:03:21,717: INFO : common : yaml file : config/config.yaml loaded successfully.]
[2025-09-02 10:03:21,719: INFO : common : yaml file : params.yaml loaded successfully.]
[2025-09-02 10:03:21,720: INFO : common : Created directory at : artifacts]
[2025-09-02 10:03:21,721: INFO : common : Created directory at : artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_299889/2355301370.py:42: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/home/rui/miniconda3/envs/textS/lib/python3.11/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used

Step,Training Loss,Validation Loss


/home/rui/miniconda3/envs/textS/lib/python3.11/site-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
